# Representation Learning & Generative Learning with AutoEncoders, GANs, and Diffusion Models

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf